<a href="https://colab.research.google.com/github/Murolando/first_recomm_system/blob/main/Recomm_Sys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Рекомендательная система по подбору фильму
### Рекомендательные системы бывают 3 типов:


*   По популярности (просто, но не индивидуально)
*   На основе содержания (похоже, на то, что ты делал раньше, очень просто, но новы вещи не смотрятся и не открываются)
*   Колаборативная система (пользователи и предметы ) 


---



*   Если основано на пользователях, то обыно рекомендуются товары схожих пользователей
*   Если основано на предмете рекомендации, то обычно сравнивают схожести этих предметов, т.е. близость определяется, но основе всех пользователей (обычно косинусное сходство)  - тут будет реализованно именно так. Но затем я буду улучшать и усложнять данную модель.


Реализовано на основе этой статьи: [https://www.dmitrymakarov.ru/intro/recommender-17/]

Векторная модель, кста

### Подготовка


In [92]:
import pandas as pd
import numpy as np

from sklearn.neighbors import NearestNeighbors


# Для работы с разреженными матрицам
from scipy.sparse import csr_matrix

In [93]:
df_movies = pd.read_csv('/content/drive/MyDrive/ML Pr/Recoms_of_films/movies.csv')
df_rates = pd.read_csv('/content/drive/MyDrive/ML Pr/Recoms_of_films/ratings.csv')


In [94]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


In [95]:
df_rates.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [96]:
# сводная таблица
user_item_matrix = df_rates.pivot(index = 'movieId', columns = 'userId', values= 'rating')
user_item_matrix.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,NaN,NaN,4.0,NaN,4.5,NaN,NaN,NaN,...,4.0,NaN,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
2,NaN,NaN,NaN,NaN,NaN,4.0,NaN,4.0,NaN,NaN,...,NaN,4.0,NaN,5.0,3.5,NaN,NaN,2.0,NaN,NaN
3,4.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN


In [97]:
# пропуски NaN нужно преобразовать в нули
# параметр inplace = True опять же поможет сохранить результат
user_item_matrix.fillna(0, inplace = True)
user_item_matrix.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9724 entries, 1 to 193609
Columns: 610 entries, 1 to 610
dtypes: float64(610)
memory usage: 45.3 MB


#### Уберем всех пользователей, которые ставят мало оценок и все фильмы, у которых также мало оценок

In [98]:
# вначале сгруппируем (объединим) пользователей, возьмем только столбец rating 
# и посчитаем, сколько было оценок у каждого пользователя
users_votes = df_rates.groupby('userId')['rating'].agg('count')

# сделаем то же самое, только для фильма
movies_votes = df_rates.groupby('movieId')['rating'].agg('count')

In [99]:
# теперь создадим фильтр (mask)
user_mask = users_votes[users_votes > 50].index
movie_mask = movies_votes[movies_votes > 10].index

In [100]:
# применим фильтры и отберем фильмы с достаточным количеством оценок
user_item_matrix = user_item_matrix.loc[movie_mask,:]

# а также активных пользователей
user_item_matrix = user_item_matrix.loc[:,user_mask]

In [101]:
user_item_matrix.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 2121 entries, 1 to 187593
Columns: 378 entries, 1 to 610
dtypes: float64(378)
memory usage: 6.1 MB


#### Данные существенно сократились, также в нашей матрице очень много нулей, такая матрица называется разреженной, к тому же наша матрица с высокой размерностью, и мы будем очень долго по ней считать, как для этого мы и импортировали библиотеку **csr_matrix**


#### Следуюущая функция, оставит только не нулевые значений, для быстрой работы алгоритма 

In [102]:
csr_data = csr_matrix(user_item_matrix.values)
print(csr_data)

  (0, 0)	4.0
  (0, 3)	4.5
  (0, 6)	2.5
  (0, 8)	4.5
  (0, 9)	3.5
  (0, 10)	4.0
  (0, 12)	3.5
  (0, 16)	3.0
  (0, 19)	3.0
  (0, 20)	3.0
  (0, 25)	5.0
  (0, 28)	5.0
  (0, 29)	4.0
  (0, 31)	3.0
  (0, 34)	5.0
  (0, 38)	5.0
  (0, 39)	4.0
  (0, 40)	4.0
  (0, 41)	2.5
  (0, 43)	4.5
  (0, 46)	0.5
  (0, 47)	4.0
  (0, 50)	2.5
  (0, 53)	4.0
  (0, 55)	3.0
  :	:
  (2118, 205)	4.0
  (2118, 345)	1.5
  (2118, 357)	4.0
  (2118, 369)	4.5
  (2119, 37)	3.5
  (2119, 62)	3.0
  (2119, 98)	0.5
  (2119, 127)	4.5
  (2119, 156)	4.5
  (2119, 236)	0.5
  (2119, 256)	4.5
  (2119, 317)	2.0
  (2119, 345)	2.0
  (2119, 357)	5.0
  (2119, 365)	3.5
  (2120, 37)	4.0
  (2120, 62)	5.0
  (2120, 146)	2.5
  (2120, 155)	4.5
  (2120, 156)	5.0
  (2120, 186)	5.0
  (2120, 205)	4.0
  (2120, 236)	3.0
  (2120, 317)	3.5
  (2120, 357)	4.0


In [103]:
# остается только сбросить индекс с помощью reset_index()
# это необходимо для удобства поиска фильма по индексу, ведь мы удалили много индексов, и чтобы было проще работать
user_item_matrix = user_item_matrix.rename_axis(None, axis = 1).reset_index()
user_item_matrix.head()

,movieId,1,4,6,7,10,11,15,16,17,...,600,601,602,603,604,605,606,607,608,610
0,1,4.0,0.0,0.0,4.5,0.0,0.0,2.5,0.0,4.5,...,2.5,4.0,0.0,4.0,3.0,4.0,2.5,4.0,2.5,5.0
1,2,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,0.0,4.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0
2,3,4.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
3,5,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.5,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0
4,6,4.0,0.0,4.0,0.0,0.0,5.0,0.0,0.0,0.0,...,0.0,0.0,3.0,4.0,3.0,0.0,0.0,0.0,0.0,5.0


### Обучение модели

#### Для наших целей нам достаточно измерить расстояние между объектами. В этом нам поможет класс машинного обучения без учителя NearestNeighbors. Т.к нам необходимо будет только посчитать расстояния  мы используем алгоритм без учителя

In [104]:
# воспользуемся классом NearestNeighbors для поиска расстояний
knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute', n_neighbors = 20, n_jobs = -1)

# обучим модель
knn.fit(csr_data)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=20)

metric = ‘cosine’: выбираем способ измерения расстояния, в нашем случае это будет косинусное сходство

algorithm = ‘brute’: предполагает, что мы будем искать решение методом полного перебора (brute force search), в данном случае пространство решений позволяет перебрать все варианты

n_neighbors = 20: по скольким соседям ведется обучение

n_jobs = -1: в этом случае предполагается, что вычисления будут вестись на всех свободных ядрах процессора

### Составления рекомендаций

In [105]:
# количество рекомендаций
recoms = 10
# Фильм на основе которого мы ищем и считаем
search = 'Matrix'

In [106]:
# для начала найдем фильм в заголовках датафрейма movies
movie_search = df_movies[df_movies['title'].str.contains(search)]
movie_search

,movieId,title,genres
1939,2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller
4351,6365,"Matrix Reloaded, The (2003)",Action|Adventure|Sci-Fi|Thriller|IMAX
4639,6934,"Matrix Revolutions, The (2003)",Action|Adventure|Sci-Fi|Thriller|IMAX


In [107]:
# вариантов может быть несколько, для простоты всегда будем брать первый вариант
# через iloc[0] мы берем первую строку столбца ['movieId']
movie_id = movie_search.iloc[0]['movieId']

# далее по индексу фильма в датасете movies найдем соответствующий индекс
# в матрице предпочтений
movie_id = user_item_matrix[user_item_matrix['movieId'] == movie_id].index[0]
movie_id

901

#### С помощью нашей системы, ищем ближайшией фильмы и расстояния до них, соседей берем +1 т.к алгоритм также рассматривает расстояние и в самом себе

In [108]:
# теперь нужно найти индексы и расстояния фильмов, которые похожи на наш запрос
# воспользуемся методом kneighbors()
distances, indices = knn.kneighbors(csr_data[movie_id], n_neighbors = recoms + 1)

In [109]:
# индексы рекомендованных фильмов
indices

array([[ 901, 1002,  442,  454,  124,  735,  954, 1362, 1157, 1536,  978]])

In [110]:
# расстояния до них
distances

array([[0.        , 0.22982441, 0.25401128, 0.27565617, 0.27760886,
        0.28691008, 0.29111012, 0.31393358, 0.31405926, 0.31548004,
        0.31748544]])

In [111]:
# уберем лишние измерения через squeeze() и преобразуем массивы в списки с помощью tolist()
indices_list = indices.squeeze().tolist()
distances_list = distances.squeeze().tolist()

In [112]:
indices_list

[901, 1002, 442, 454, 124, 735, 954, 1362, 1157, 1536, 978]

In [113]:
# далее с помощью функций zip и list преобразуем наши списки  в набор кортежей (tuple)
indices_distances = list(zip(indices_list, distances_list))
indices_distances


[(901, 0.0),
 (1002, 0.22982440568634488),
 (442, 0.25401128310081567),
 (454, 0.27565616686043737),
 (124, 0.2776088577731709),
 (735, 0.2869100842838125),
 (954, 0.2911101181714415),
 (1362, 0.31393358217709477),
 (1157, 0.31405925934381695),
 (1536, 0.3154800434449465),
 (978, 0.31748544046311844)]

In [114]:
# остается отсортировать список по расстояниям через key = lambda x: x[1] (то есть по второму элементу)
indices_distances_sorted = sorted(indices_distances, key = lambda x: x[1])
# Т.к 0 это сама матрица 
indices_distances_sorted = indices_distances_sorted[1:]
indices_distances_sorted

[(1002, 0.22982440568634488),
 (442, 0.25401128310081567),
 (454, 0.27565616686043737),
 (124, 0.2776088577731709),
 (735, 0.2869100842838125),
 (954, 0.2911101181714415),
 (1362, 0.31393358217709477),
 (1157, 0.31405925934381695),
 (1536, 0.3154800434449465),
 (978, 0.31748544046311844)]

#### Теперь надо вывести все эти фильмы вместе с дистанциями 

In [115]:
def films_and_dists(indices_distances_sorted):
  names_and_dists = []
  for i in indices_distances_sorted:
    # искать movieId в матрице предпочтений
    matrix_movie_id = user_item_matrix.iloc[i[0]]['movieId']

    # выяснять индекс этого фильма в датафрейме movies
    id = df_movies[df_movies['movieId'] == matrix_movie_id].index


    name = df_movies.iloc[id]['title'].values[0]
    dist = i[1]
    names_and_dists.append((name,dist))
  return names_and_dists

films_and_dists(indices_distances_sorted)

[('Fight Club (1999)', 0.22982440568634488),
 ('Star Wars: Episode V - The Empire Strikes Back (1980)',
  0.25401128310081567),
 ('Star Wars: Episode VI - Return of the Jedi (1983)', 0.27565616686043737),
 ('Star Wars: Episode IV - A New Hope (1977)', 0.2776088577731709),
 ('Saving Private Ryan (1998)', 0.2869100842838125),
 ('Sixth Sense, The (1999)', 0.2911101181714415),
 ('Lord of the Rings: The Fellowship of the Ring, The (2001)',
  0.31393358217709477),
 ('Gladiator (2000)', 0.31405925934381695),
 ('Lord of the Rings: The Return of the King, The (2003)', 0.3154800434449465),
 ('American Beauty (1999)', 0.31748544046311844)]

### Улучшения

#### Попробуем вместо уменьшения датасета, для повышения производительности, просто поменяем алгоритм с грубого перебора на BAll Tree, и тогда мы сможем оставить исходный датасет

Было:

Int64Index: 2121 entries, 1 to 187593
Columns: 378 entries, 1 to 610

Стало:

Int64Index: 9724 entries, 1 to 193609 
Columns: 610 entries, 1 to 610



In [116]:
full_matrix = df_rates.pivot(index = 'movieId', columns = 'userId', values= 'rating')
full_matrix.fillna(0, inplace = True)
full_matrix.info()
full_matrix = full_matrix.rename_axis(None, axis = 1).reset_index()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9724 entries, 1 to 193609
Columns: 610 entries, 1 to 610
dtypes: float64(610)
memory usage: 45.3 MB


In [117]:
csr_full_data = csr_matrix(user_item_matrix.values)
#print(csr_full_data)

In [118]:
csr_full_data

<2121x379 sparse matrix of type '<class 'numpy.float64'>'
	with 75014 stored elements in Compressed Sparse Row format>

In [119]:
from sklearn.neighbors import BallTree
# воспользуемся классом BallTree для поиска расстояний
knn_ball_tree = BallTree(full_matrix)

# Выведем айди и расстояния до 10 ближайших соседей

dist, ind = knn_ball_tree.query(full_matrix[:1],k=10)

print(dist)

print(ind)

[[ 0.         55.16112762 56.20720594 56.62375826 57.36723804 57.39337941
  57.6432997  57.72347876 57.75162335 57.87918451]]
[[ 0  1  4  2  6 18  9  8  5 10]]


In [120]:
# knn = NearestNeighbors(metric = 'cosine', algorithm = 'ball_tree', n_neighbors = 20, n_jobs = -1)
# knn.fit(csr_data)

In [121]:
indices_list_BT = ind.squeeze().tolist()
distances_list_BT = dist.squeeze().tolist()
indices_distances_BT = list(zip(indices_list_BT, distances_list_BT))

indices_distances_sorted_full_matrix = sorted(indices_distances_BT, key = lambda x: x[1])
indices_distances_sorted_full_matrix = indices_distances_sorted_full_matrix[1:]
indices_distances_sorted_full_matrix

[(1, 55.161127617190715),
 (4, 56.2072059437222),
 (2, 56.62375826453062),
 (6, 57.367238037053866),
 (18, 57.39337940912697),
 (9, 57.64329969736292),
 (8, 57.723478758647246),
 (5, 57.75162335380712),
 (10, 57.879184513951124)]

In [122]:
names_and_dists = []
for i in indices_distances_sorted_full_matrix:
  # искать movieId в матрице предпочтений
  matrix_movie_id = full_matrix.iloc[i[0]]['movieId']

  # выяснять индекс этого фильма в датафрейме movies
  id = df_movies[df_movies['movieId'] == matrix_movie_id].index
  name = df_movies.iloc[id]['title'].values[0]
  dist = i[1]
  names_and_dists.append((name,dist))
names_and_dists

[('Jumanji (1995)', 55.161127617190715),
 ('Father of the Bride Part II (1995)', 56.2072059437222),
 ('Grumpier Old Men (1995)', 56.62375826453062),
 ('Sabrina (1995)', 57.367238037053866),
 ('Ace Ventura: When Nature Calls (1995)', 57.39337940912697),
 ('GoldenEye (1995)', 57.64329969736292),
 ('Sudden Death (1995)', 57.723478758647246),
 ('Heat (1995)', 57.75162335380712),
 ('American President, The (1995)', 57.879184513951124)]

##### Такие разные данные получились, потому как в первом случае, у нас были не все фильмы + мы использовали косинусное сходство, а сейчас BallTree использовало метрику меньковского

### P.S Оценить точность таких систем слонжо, т.к нет правильных ответов, а факт того, что ответы контринтуитивны, мало чего значит, ибо контринтуитивность != аргумент, единственный вариант это использование A/B тестов
